In [2]:
import json
import math
from datetime import datetime

# Конфигурация
OUTPUT_FILE = "bus_arrivals_nearest1.json"  # Файл для записи результатов

def haversine_distance(lon1, lat1, lon2, lat2):
    """Упрощенное расстояние между точками (в градусах)"""
    # Радиус Земли в метрах
    R = 6371000

    # Преобразование градусов в радианы
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Разница координат
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    # Формула Гаверсина
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

def load_track_and_stops(track_file, stops_file):
    """
    Загружает данные трека и остановок из JSON-файлов и преобразует строковые координаты в числа.

    Args:
        track_file (str): Путь к JSON-файлу, содержащему данные трека.
        stops_file (str): Путь к JSON-файлу, содержащему данные остановок.

    Returns:
        tuple: Кортеж, содержащий два массива:
               - track (list): Массив точек трека.
               - stops (list): Массив остановок.
               Возвращает (None, None), если произошла ошибка при загрузке файлов.
    """
    try:
        with open(track_file, 'r') as f:
            track = json.load(f)

    except FileNotFoundError:
        print(f"Ошибка: Файл трека не найден: {track_file}")
        return None, None
    except json.JSONDecodeError:
        print(f"Ошибка: Неверный формат JSON в файле трека: {track_file}")
        return None, None
    except Exception as e:
        print(f"Ошибка при загрузке файла трека: {e}")
        return None, None

    try:
        with open(stops_file, 'r') as f:
            stops_data = json.load(f)
            # Извлекаем остановки из структуры JSON
            stops = stops_data #['routes'][0]['stops']
    except FileNotFoundError:
        print(f"Ошибка: Файл остановок не найден: {stops_file}")
        return None, None
    except json.JSONDecodeError:
        print(f"Ошибка: Неверный формат JSON в файле остановок: {stops_file}")
        return None, None
    except Exception as e:
        print(f"Ошибка при загрузке файла остановок: {e}")
        return None, None

    return track, stops

def find_nearest_stop(lon, lat, direction_id, stops):
    """Находит ближайшую остановку к заданной точке"""
    nearest_stop = None
    min_distance = 50

    if (direction_id=="1549259685526025"):
        route1 = "Обратно от ост. Топкинский микрорайон"
    else:
        route1 = "До ост. Топкинский микрорайон"

    print(route1)

    for route in stops:
        print(route)
        if (route == route1):
            for stop in stops:
                stop_lon = stop['coordinates']['lon']
                stop_lat = stop['coordinates']['lat']
                distance = haversine_distance(lon, lat, stop_lon, stop_lat)

                if distance < min_distance:
                    min_distance = distance
                    nearest_stop = stop

    return nearest_stop, min_distance

def save_arrival(arrival):
    """Сохраняет информацию о прибытии в файл"""
    try:
        with open(OUTPUT_FILE, 'r') as f:
            existing_arrivals = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        existing_arrivals = []

    existing_arrivals.append(arrival)

    with open(OUTPUT_FILE, 'w') as f:
        json.dump(existing_arrivals, f, indent=2, ensure_ascii=False) #Чтобы читалось по русски

def process_track(track, stops):
    """Обрабатывает трек и определяет прибытия на ближайшие остановки"""
    if not track or not stops:
        print("Ошибка: Нет данных трека или остановок для обработки.")
        return

    for i in range(len(track)):
        point = track[i]

        #Проверка что х и у есть
        if 'x' not in point or 'y' not in point:
            continue

        lon, lat = point['x'], point['y']
        timestamp = point['track_timestamp']
        bus_id = point['device_id']
        direction_id = point['direction_id']

        nearest_stop, distance = find_nearest_stop(lon, lat, direction_id, stops)

        if nearest_stop:
            arrival_info = {
                'bus_id': bus_id,
                'direction_id': direction_id,
                'stop_name': nearest_stop['name'],
                'timestamp': timestamp,
                'datetime': datetime.fromtimestamp(timestamp).isoformat(),
                'bus_coordinates': {'lat': lat, 'lon': lon},
                'stop_coordinates': nearest_stop['coordinates'],
                'distance': distance
            }
            save_arrival(arrival_info)
            print(f"Автобус {bus_id} прибыл на остановку {nearest_stop['name']} (расстояние: {distance:.4f})")

# Пример использования
track_file = "buses.json"  # Замените на путь к вашему файлу трека
stops_file = "output.json"  # Замените на путь к вашему файлу остановок

track, stops = load_track_and_stops(track_file, stops_file)

if track and stops:
    process_track(track, stops)
    print(f"Данные о прибытиях записаны в файл {OUTPUT_FILE}")
else:
    print("Не удалось загрузить данные. Проверьте пути к файлам.")


Обратно от ост. Топкинский микрорайон
routes
Обратно от ост. Топкинский микрорайон
routes
Обратно от ост. Топкинский микрорайон
routes
Обратно от ост. Топкинский микрорайон
routes
До ост. Топкинский микрорайон
routes
До ост. Топкинский микрорайон
routes
До ост. Топкинский микрорайон
routes
До ост. Топкинский микрорайон
routes
До ост. Топкинский микрорайон
routes
До ост. Топкинский микрорайон
routes
Обратно от ост. Топкинский микрорайон
routes
Обратно от ост. Топкинский микрорайон
routes
Обратно от ост. Топкинский микрорайон
routes
Обратно от ост. Топкинский микрорайон
routes
До ост. Топкинский микрорайон
routes
До ост. Топкинский микрорайон
routes
До ост. Топкинский микрорайон
routes
До ост. Топкинский микрорайон
routes
До ост. Топкинский микрорайон
routes
До ост. Топкинский микрорайон
routes
Обратно от ост. Топкинский микрорайон
routes
Обратно от ост. Топкинский микрорайон
routes
Обратно от ост. Топкинский микрорайон
routes
До ост. Топкинский микрорайон
routes
До ост. Топкинский микро